In [ ]:
# Relevant imports

import numpy as np
import pandas as pd
import evaluate
import torch
import json
from datasets import Dataset, DatasetDict
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from sklearn.metrics import accuracy_score, f1_score
from torch.nn import CrossEntropyLoss

from preprocessing import climateBUG_reduce_rows



In [ ]:
# Checks whether the GPU is available for use
print(f"is GPU available: {torch.cuda.is_available()}")

is GPU available: True


In [ ]:
# Preprocessing occurs in the preprocessing.py file. 
# In this case, the training and testing datasets are reduced to 10000 and 1000 rows respectively

train_df = pd.read_json("/home/lukeg/Documents/VS_code/fine_tuning/lxg406/climate_relatedness_classification/data/climateBUG/climateBUG-training-dataset.json")
preprocessed_training_df = climateBUG_reduce_rows(train_df, rows=10000)

test_df = pd.read_json("/home/lukeg/Documents/VS_code/fine_tuning/lxg406/climate_relatedness_classification/data/climateBUG/climateBUG-testing-dataset.json")
preprocessed_testing_df = climateBUG_reduce_rows(test_df, rows=1000)


                                              statement  label
0     2015REGISTRATION DOCUMENT ANNUAL FINANCIAL REP...      0
1     Retail     banking     achieved     a     stro...      0
2     Crédit   Agricole   Assurances   captured  fur...      0
3     Our  business  is  also  exposed  to  the  ups...      0
4     Crédit  Agricole  assumed  its   responsibilit...      0
...                                                 ...    ...
9995  Other general administrative expenses (Note 33...      0
9996  Other financial instruments not measured at fa...      0
9997             Revaluation gains/(losses) 191 (1,054)      0
9998  Final balance at 31.12.2014 5,714,956 12,032,8...      0
9999                Tangible assets (218,340) (249,701)      0

[10000 rows x 2 columns]
                                              statement  label
0     2015REGISTRATION DOCUMENT ANNUAL FINANCIAL REP...      0
1     Retail     banking     achieved     a     stro...      0
2     Crédit   Agricole   Ass

In [ ]:
# Simple checks that the dataset is working correctly
preprocessed_training_df.loc[0]

statement    2015REGISTRATION DOCUMENT ANNUAL FINANCIAL REP...
label                                                        0
Name: 0, dtype: object

In [5]:
preprocessed_testing_df.loc[0]

statement    4CRÉDIT AGRICOLE S.A. – 2020-2021 INTEGRATED R...
label                                                        0
Name: 0, dtype: object

In [ ]:
# Datasets are converted from pandas dataframes into HuggingFace dataset objects
training_dataset = Dataset.from_pandas(preprocessed_training_df)
testing_dataset = Dataset.from_pandas(preprocessed_testing_df)
training_dataset[0]

{'statement': '2015REGISTRATION DOCUMENT ANNUAL FINANCIAL REPORT2 CRÉDIT AGRICOLE S.A.      2015 REGISTRATION DOCUMENT MESSAGE FROM THE CHAIRMAN AND THE CHIEF EXECUTIVE OFFICER MESSAGE FROM THE CHAIRMANAND THE CHIEF EXECUTIVE OFFICERPHILIPPE BRASSAC, CHIEF EXECUTIVE OFFICER AND DOMINIQUE LEFEBVRE, CHAIRMAN OF CRÉDIT AGRICOLE S.A.A robust Group delivering sustainable performanceLeading  a  bank  with  a  EUR  698  billion   loan   book(1)',
 'label': 0}

In [ ]:
# Dataset is shuffled using specified seed
seed=11
training_dataset = training_dataset.shuffle(seed=seed)
testing_dataset = testing_dataset.shuffle(seed=seed)

In [ ]:
# Imports tokenizer from HuggingFace
tokenizer = AutoTokenizer.from_pretrained("climatebert/distilroberta-base-climate-f")

/home/lukeg/ClimatEnv/lib/python3.12/site-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [ ]:
# Custom Tokenize function allowing the editing of padding and text.
# This is more relevant when working with climate-FEVER
def custom_tokenize(examples):
    tokenized_output = tokenizer(
        text=examples["statement"],
        max_length=512, 
        padding="max_length", 
        truncation=True)

    return tokenized_output

tokenized_training_dataset = training_dataset.map(custom_tokenize, batched=True)
tokenized_testing_dataset = testing_dataset.map(custom_tokenize, batched=True)

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [ ]:
# Prints information about the dataset
tokenized_training_dataset

Dataset({
    features: ['statement', 'label', 'input_ids', 'attention_mask'],
    num_rows: 10000
})

In [ ]:
# Checks information
print(tokenized_training_dataset[0]["statement"])
print(tokenized_training_dataset[0]["label"])
print(tokenized_training_dataset[0]["input_ids"])
print(tokenized_training_dataset[0]["attention_mask"])

print(tokenized_testing_dataset[9]["statement"])
print(tokenized_testing_dataset[9]["label"])
print(tokenized_testing_dataset[9]["input_ids"])
print(tokenized_testing_dataset[9]["attention_mask"])

Refer to https://sustainabledevelopment.un.org/ for more information on the Sustainable Development Goals.
1
[0, 48310, 7, 1205, 640, 50284, 27013, 4, 879, 4, 1957, 73, 13, 55, 335, 15, 5, 20586, 2717, 20638, 4, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

In [ ]:
# Model is accessed from HuggingFace. 
# Ignore_mismatched is only necessary for climateBUG and climateBERT-detector when working with climate-FEVER
model = AutoModelForSequenceClassification.from_pretrained(
    "climatebert/distilroberta-base-climate-f",
    num_labels=2,
    # ignore_mismatched_sizes=True
).to("cuda")

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at climatebert/distilroberta-base-climate-f and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Ensures the GPU is being used
print(next(model.parameters()).device)

cuda:0


In [ ]:
# Enables gradient checkpointing for memory efficiency
model.gradient_checkpointing_enable()

In [ ]:
# Custom metrics function, which ensures the trainer will evaluate using the desired metrics.
# By default, the trainer only evaluates on Loss, so F1 and accuracy are added
def calculate_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=1)
    
    accuracy = accuracy_score(labels, preds)
    f1 = f1_score(labels, preds, average="weighted")

    return {
        "accuracy": accuracy,
        "f1_score": f1,
    }

In [ ]:
# Hyperparameters are set, along with other parameters needed for working with HuggingFace's trainer object
training_args = TrainingArguments(
    output_dir="./results/climateBERT-base",
    per_device_train_batch_size=8,
    num_train_epochs=3,
    evaluation_strategy="steps",
    eval_steps=200,
    logging_strategy="steps",
    logging_steps=50,
    save_strategy="no",
    save_steps=500,
    fp16=True,                          # Use 16-bit floating point instead of 32 - makes computation faster
    warmup_ratio=0.05,                    # Allows the model to adapt a little
    # gradient_accumulation_steps=2       # Might help with OOM errors, if we have them
    learning_rate=3e-5,
    push_to_hub=False,
)

In [ ]:
# Trainer object is created and arguments passed
trainer  = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_training_dataset,
    tokenizer=tokenizer,
    eval_dataset=tokenized_testing_dataset,
    compute_metrics=calculate_metrics,
)

In [ ]:
# Training (fine-tuning) commences
trainer.train()

  0%|          | 0/3750 [00:00<?, ?it/s]

{'loss': 0.6854, 'grad_norm': 1.2652164697647095, 'learning_rate': 7.978723404255319e-06, 'epoch': 0.04}
{'loss': 0.5318, 'grad_norm': inf, 'learning_rate': 1.579787234042553e-05, 'epoch': 0.08}
{'loss': 0.3276, 'grad_norm': 39.72299575805664, 'learning_rate': 2.3617021276595744e-05, 'epoch': 0.12}
{'loss': 0.3478, 'grad_norm': 4.295405864715576, 'learning_rate': 2.9915777653003933e-05, 'epoch': 0.16}


  0%|          | 0/125 [00:00<?, ?it/s]

{'eval_loss': 0.3082336485385895, 'eval_accuracy': 0.877, 'eval_f1_score': 0.8738757277551652, 'eval_runtime': 8.8726, 'eval_samples_per_second': 112.706, 'eval_steps_per_second': 14.088, 'epoch': 0.16}
{'loss': 0.2264, 'grad_norm': 2.008941888809204, 'learning_rate': 2.9494665918023583e-05, 'epoch': 0.2}
{'loss': 0.3023, 'grad_norm': 8.739518165588379, 'learning_rate': 2.908197641774284e-05, 'epoch': 0.24}
{'loss': 0.2715, 'grad_norm': 10.10730266571045, 'learning_rate': 2.8660864682762495e-05, 'epoch': 0.28}
{'loss': 0.3123, 'grad_norm': 5.148792266845703, 'learning_rate': 2.8239752947782145e-05, 'epoch': 0.32}


  0%|          | 0/125 [00:00<?, ?it/s]

...

  0%|          | 0/125 [00:00<?, ?it/s]

{'eval_loss': 0.4493371546268463, 'eval_accuracy': 0.905, 'eval_f1_score': 0.9055759681791999, 'eval_runtime': 9.2038, 'eval_samples_per_second': 108.65, 'eval_steps_per_second': 13.581, 'epoch': 2.72}
{'loss': 0.1577, 'grad_norm': 0.09865036606788635, 'learning_rate': 2.5603593486805164e-06, 'epoch': 2.76}
{'loss': 0.1075, 'grad_norm': 0.27391910552978516, 'learning_rate': 2.1392476137001685e-06, 'epoch': 2.8}
{'loss': 0.1396, 'grad_norm': 1.0460302829742432, 'learning_rate': 1.7181358787198205e-06, 'epoch': 2.84}
{'loss': 0.1128, 'grad_norm': 13.918270111083984, 'learning_rate': 1.2970241437394723e-06, 'epoch': 2.88}


  0%|          | 0/125 [00:00<?, ?it/s]

{'eval_loss': 0.4513951539993286, 'eval_accuracy': 0.905, 'eval_f1_score': 0.9055759681791999, 'eval_runtime': 9.228, 'eval_samples_per_second': 108.365, 'eval_steps_per_second': 13.546, 'epoch': 2.88}
{'loss': 0.164, 'grad_norm': 0.03540243208408356, 'learning_rate': 8.759124087591241e-07, 'epoch': 2.92}
{'loss': 0.238, 'grad_norm': 0.02691902592778206, 'learning_rate': 4.5480067377877594e-07, 'epoch': 2.96}
{'loss': 0.1737, 'grad_norm': 0.10168439894914627, 'learning_rate': 3.368893879842785e-08, 'epoch': 3.0}
{'train_runtime': 1380.8897, 'train_samples_per_second': 21.725, 'train_steps_per_second': 2.716, 'train_loss': 0.21658149445851643, 'epoch': 3.0}


TrainOutput(global_step=3750, training_loss=0.21658149445851643, metrics={'train_runtime': 1380.8897, 'train_samples_per_second': 21.725, 'train_steps_per_second': 2.716, 'total_flos': 3974021959680000.0, 'train_loss': 0.21658149445851643, 'epoch': 3.0})

In [ ]:
# Saves the model. This has been omitted since I produce no model worth keeping
# trainer.save_model("./results/climateBERT-base/climateBUG/first_run")

In [ ]:
# Evaluation commences. The results are then saved into a file specific to the seed used. 
# Seeds are changed every run, but all models are evaluated on the same set of seeds
metrics = trainer.evaluate()
with open(f"./results/climateBERT-base/eval_metrics_seed{seed}.json", "w") as output_file:
    json.dump(metrics, output_file)

  0%|          | 0/125 [00:00<?, ?it/s]